# Assigning Regions to Sale Representatives at Pfizer Turkey 


In [5]:
import pandas as pd
import numpy as np

assignment = pd.read_csv("assignment.csv")
distances = pd.read_csv("brick_rp_distances.csv")
workload = pd.read_csv("bricks_index_values.csv")

In [10]:
print(workload.columns)
workload["index_value"].sum()

Index(['brick', 'index_value'], dtype='object')


np.float64(3.9999999999999996)